# 금융 데이터 Pinecone 임베딩 저장

이 노트북은 금융 관련 질의-계획 데이터를 OpenAI 임베딩을 사용해 Pinecone에 저장합니다.  

### 1.셋업

In [1]:
import json
from langchain_pinecone import PineconeVectorStore
from typing import List, Dict
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
import os
from langchain_core.documents import Document

load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

options = {
    "planner":{
    "file":"planner_shots.json",
    "index_name":"aicfo-planner",
    "documentKeys":["task", "plan"]},
    "trsc_nl2sql":{
    "file":"transaction_nl2sql_shots.json",
    "index_name":"aicfo-transaction-nl2sql",
    "documentKeys":["질문", "SQL"]}
}

# 이 부분을 수정해 주세요.
option = "planner"
# option = "trsc_nl2sql"


### 2.데이터 로드

In [2]:
def load_data(file_path: str) -> List[Dict]:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

data = load_data(options[option]["file"])
print(data)

[{'task': '고정입금내역 보여줘', 'plan': 'Plan: 최근 3개월 간의 거래 데이터를 가져옵니다. 오늘이 2024년 11월 20일이므로, 최근 3개월은 2024년 8월 20일부터 2024년 11월 19일까지입니다.\n#E1 = load_transaction_data["20240820", "20241119"]\nPlan: 3개월 기준 고정 입금 내역을 검색합니다.\n#E2 = transaction_sql["3개월 기준 고정입금 내역을 찾아줘", "#E1"]'}, {'task': '대한전자 돈 들어왔어?', 'plan': 'Plan: 지난 1개월 간의 거래 데이터를 가져옵니다.\n#E1 = load_transaction_data["20230801","20230901"]\nPlan: 거래 데이터에서 적요가 "대한전자"인 것 중에 가장 최신 데이터를 가져옵니다.\n#E2 = transaction_sql["적요가 대한전자인 거래 중에 가장 최신 것만 보여줘", "#E1"]'}, {'task': '월평균 순현금흐름 보여줘', 'plan': 'Plan: 지난 3개월의 거래 데이터를 가져옵니다.\n#E1 = load_transaction_data["20230701","20230930"]\nPlan: 거래 데이터에서 "월평균 순현금흐름"을 계산합니다.\n#E2 = transaction_sql["월평균 순현금흐름을 계산해줘", "#E1"]'}, {'task': '24년 급여 내역 보여줘', 'plan': 'Plan: 2024년의 모든 입출금 내역을 가져옵니다.\n#E1 = load_transaction_data["20240101", "20241231"]\nPlan: 적요가 급여인 거래내역을 찾습니다.\n#E2 = transaction_sql["적요가 급여인 거래내역 찾아줘", "#E1"]'}, {'task': '대한전자 월 입금 얼마씩 들어와?', 'plan': 'Plan: 최근 1년 간의 거래 데이터를 가져옵니다.\n#E1 = load_transaction_d

### 3.파인콘 인덱스 및 openai embedding 가져오기

In [3]:
# OpenAI 임베딩 초기화
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)

index_name = options[option]["index_name"]  

# Pinecone init
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

/var/folders/hq/90195vnj7n1gt1ymt668yjqm0000gn/T/ipykernel_8701/4075036742.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


### 4.이제 다큐먼트를 올려봅시다.

In [4]:
# 텍스트와 메타데이터를 만듭니다.
documents = []
pageKey = options[option]["documentKeys"][0]
dataKey = options[option]["documentKeys"][1]

for item in data:
    documents.append(Document(page_content=item[pageKey], metadata={dataKey:item[dataKey]}))


# 벡터스토어에 저장합니다.
vectorstore.delete(delete_all=True)
ids = vectorstore.add_documents(
        documents=documents
)

print(len(ids))

51


### 5.테스트 쿼리 실행

데이터가 제대로 저장되었는지 확인하기 위한 테스트 쿼리를 실행해봅시다.

In [5]:
results = vectorstore.similarity_search(
    "대출 잔액",
    k=2
)
print(vectorstore)
print(results)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

[]
